# Inference with Fine-Tuned EfficientNet Model

## Introduction

In this notebook, we will perform inference using a fine-tuned EfficientNet model on a dataset related to lumbar spine degenerative classification. The EfficientNet architecture, known for its efficiency and accuracy, has been pre-trained on a large dataset and fine-tuned to adapt to our specific classification task.

### Objectives

1. **Set Up Environment**: Import necessary libraries and load the dataset.
2. **Load the Fine-Tuned Model**: Load the previously saved model to make predictions on the test dataset.
3. **Process Test Images**: Prepare the test images using appropriate transformations.
4. **Make Predictions**: Utilize the model to predict the classes of the test images.
5. **Prepare Submission**: Format the predictions for submission in a required format.

### Prerequisites

- Basic knowledge of PyTorch and deep learning concepts.
- Understanding of model fine-tuning and inference processes.

By the end of this notebook, you will have a clear understanding of how to utilize a fine-tuned model for making predictions on unseen data and preparing those predictions for submission.

Let's get started!


In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd 

### Exploring the Test Dataset Descriptions

To understand the structure and contents of the `test_description` DataFrame, we use the `info()` method provided by pandas. This method summarizes important details about the DataFrame, including the number of entries, column names, non-null counts, and data types.


In [6]:
# the path to the dataset

train_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/'

test_description = pd.read_csv(train_path + 'test_series_descriptions.csv')
submission = pd.read_csv(train_path + 'sample_submission.csv')

In [7]:
test_description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   study_id            3 non-null      int64 
 1   series_id           3 non-null      int64 
 2   series_description  3 non-null      object
dtypes: int64(2), object(1)
memory usage: 200.0+ bytes


#### Summary of the DataFrame

The output reveals that the `test_description` DataFrame consists of 3 entries (or rows), indexed from 0 to 2. It contains a total of 3 columns with the following characteristics:

- **study_id**: 
  - Type: `int64` 
  - Description: This column contains unique identifiers for each study. It has 3 non-null entries, indicating that all entries are populated.

- **series_id**: 
  - Type: `int64`
  - Description: This column holds unique identifiers for each series within a study, also with 3 non-null entries.

- **series_description**: 
  - Type: `object` 
  - Description: This column contains string descriptions of each series. It similarly has 3 non-null entries, meaning that all descriptions are present.

#### Memory Usage

The DataFrame utilizes approximately 200 bytes of memory, which is a relatively small amount given the number of entries.

### Conclusion

This summary of the `test_description` DataFrame provides insight into the data we will be working with for inference. Understanding the structure and contents of the DataFrame is crucial for processing the test images and making accurate predictions.


------

### Generating Image Paths for Inference

In this section, we create a function that generates the file paths for the test images based on the structure of the dataset. This step is crucial for loading the images later, enabling us to process them and make predictions.

#### Libraries Used

- **os**: This module provides functionalities for interacting with the operating system, allowing us to create and manipulate file paths efficiently.
- **cv2**: Part of the OpenCV library, commonly used for image processing tasks. Although not used directly in this snippet, it will be helpful later.
- **matplotlib.pyplot**: This library is typically used for visualization and plotting, which may be utilized later to display images.

#### Function Overview

A function named `generate_image_paths` is defined to take a DataFrame and a base directory as input. Its purpose is to generate a list of paths for all the images related to each study and series identified in the provided DataFrame.

1. **Parameters**:
   - **DataFrame (`df`)**: This contains the `study_id` and `series_id` that identify the images.
   - **Base Directory (`data_dir`)**: This is the path to the directory where the test images are stored.

2. **Process**:
   - The function initializes an empty list to store the image paths.
   - It iterates through the `study_id` and `series_id` pairs in the DataFrame.
   - For each pair, it constructs the paths to the respective study and series directories.
   - It lists all files in the series directory and generates full paths for each image.
   - Finally, it returns the list of image paths.

#### Generating Test Image Paths

After defining the function, it is invoked to generate the paths for the test images using the `test_description` DataFrame and the specified directory path. The resulting list of paths is stored for later use.



This section ensures that all relevant test images can be accessed by dynamically generating their file paths based on the dataset's directory structure. This approach facilitates efficient loading and processing of images in the subsequent steps of our inference process.


In [8]:
import os
import cv2
import matplotlib.pyplot as plt

# Function to generate image paths based on directory structure
def generate_image_paths(df, data_dir):
    image_paths = []
    for study_id, series_id in zip(df['study_id'], df['series_id']):
        study_dir = os.path.join(data_dir, str(study_id))
        series_dir = os.path.join(study_dir, str(series_id))
        images = os.listdir(series_dir)
        image_paths.extend([os.path.join(series_dir, img) for img in images])
    return image_paths


test_image_paths = generate_image_paths(test_description, f'{train_path}/test_images')


### Condition Mapping for Medical Imaging

In this section, we define a mapping dictionary called `condition_mapping`, which is essential for linking specific imaging modalities to their corresponding medical conditions. This mapping is crucial for interpreting the results of medical images effectively.

#### Overview of the Mapping

The `condition_mapping` dictionary organizes the relationship between different types of imaging conditions (modalities) and associated diagnoses. It allows for quick access to conditions based on the type of imaging performed.

#### Structure of the Dictionary

- **Keys**: The keys represent various imaging conditions:
  - **`'Sagittal T1'`**: Refers to images taken in the sagittal plane using T1-weighted sequences.
  - **`'Axial T2'`**: Refers to images taken in the axial plane using T2-weighted sequences.
  - **`'Sagittal T2/STIR'`**: Refers to images taken in the sagittal plane using T2-weighted or Short Tau Inversion Recovery (STIR) sequences.

- **Values**: The values associated with these keys indicate the specific medical conditions:
  - For **`'Sagittal T1'`** and **`'Axial T2'`**, the values are nested dictionaries that further distinguish conditions based on left and right side afflictions:
    - **Left Conditions**: Indicate conditions affecting the left side of the spine, such as `left_neural_foraminal_narrowing` or `left_subarticular_stenosis`.
    - **Right Conditions**: Indicate conditions affecting the right side, such as `right_neural_foraminal_narrowing` or `right_subarticular_stenosis`.
  
  - For **`'Sagittal T2/STIR'`**, the value directly corresponds to a single condition, `spinal_canal_stenosis`, indicating that this imaging modality is associated with this specific diagnosis.

#### Purpose of the Mapping

The `condition_mapping` dictionary serves several important functions:

- **Organization**: It provides a structured way to relate complex imaging types with specific medical conditions, facilitating data processing.
- **Clarity**: The mapping offers a clear reference to understand which imaging modalities correlate with particular diagnoses, aiding in clinical decision-making.
- **Efficiency**: Using a dictionary allows for quick lookups of conditions based on imaging types, which enhances the efficiency of data analysis and prediction tasks.


The `condition_mapping` dictionary is a critical component of our analysis workflow. It enables us to efficiently link medical imaging modalities with their respective conditions, thereby improving our ability to interpret and analyze the results of the imaging data effectively.


In [9]:
condition_mapping = {
    'Sagittal T1': {'left': 'left_neural_foraminal_narrowing', 'right': 'right_neural_foraminal_narrowing'},
    'Axial T2': {'left': 'left_subarticular_stenosis', 'right': 'right_subarticular_stenosis'},
    'Sagittal T2/STIR': 'spinal_canal_stenosis'
}


### Expanding Image Paths into a DataFrame

In this section, we define a process for expanding the image paths from our test dataset and organizing them into a structured DataFrame. This step is crucial for facilitating data analysis and model inference.

#### Overview of the Process

1. **Function Definition**:
   - A function called `get_image_paths` is defined to retrieve image file paths from a given study and series. 

2. **Function Logic**:
   - The function constructs the path to the series directory using the `study_id` and `series_id` from the provided row.
   - It checks if the constructed path exists. If it does, it returns a list of file paths for all images in that directory. The paths are constructed using `os.path.join()`, ensuring proper handling of the filesystem structure.
   - If the series path does not exist, the function returns an empty list.

3. **Expanding Rows**:
   - An empty list called `expanded_rows` is initialized to store the expanded information for each image.
   - The code iterates through each row of the `test_description` DataFrame using `iterrows()`. For each row:
     - The function `get_image_paths` is called to retrieve the list of image paths.
     - The conditions associated with the `series_description` are obtained from the `condition_mapping` dictionary. If the conditions are a single string, they are converted into a dictionary format for uniformity.
     - The code then loops through each side (left and right) and condition pair, creating a new entry for every image path found. Each entry includes:
       - `study_id`
       - `series_id`
       - `series_description`
       - `image_path`: The path to the individual image.
       - `condition`: The corresponding condition from the mapping.
       - `row_id`: A unique identifier constructed from the `study_id` and the condition.

4. **Creating the DataFrame**:
   - After expanding all image paths, a new DataFrame called `test_df` is created from the `expanded_rows` list. This DataFrame contains comprehensive information about each image, including its associated conditions.


This section effectively constructs a structured DataFrame, `test_df`, that holds all relevant details about each image in the test dataset. By expanding the rows based on the conditions and image paths, we facilitate easier access and processing of the data for subsequent analysis and inference tasks.


----------

In [10]:
base_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'


In [11]:
def get_image_paths(row):
    series_path = os.path.join(base_path, str(row['study_id']), str(row['series_id']))
    if os.path.exists(series_path):
        return [os.path.join(series_path, f) for f in os.listdir(series_path) if os.path.isfile(os.path.join(series_path, f))]
    return []


In [12]:
expanded_rows = []
for index, row in test_description.iterrows():
    image_paths = get_image_paths(row)
    conditions = condition_mapping.get(row['series_description'], {})
    if isinstance(conditions, str):  # Single condition
        conditions = {'left': conditions, 'right': conditions}
    for side, condition in conditions.items():
        for image_path in image_paths:
            expanded_rows.append({
                'study_id': row['study_id'],
                'series_id': row['series_id'],
                'series_description': row['series_description'],
                'image_path': image_path,
                'condition': condition,
                'row_id': f"{row['study_id']}_{condition}"
            })


In [13]:
test_df = pd.DataFrame(expanded_rows)

### Updating `row_id` with Levels

In this section, we enhance the `row_id` in the `test_df` DataFrame by incorporating anatomical levels associated with spinal conditions. This improvement provides additional context to each entry in the dataset.

#### Overview of the Process

1. **Defining Levels**:
   - A list named `levels` is defined, containing the following anatomical levels commonly associated with spinal imaging:
     - `'l1_l2'`
     - `'l2_l3'`
     - `'l3_l4'`
     - `'l4_l5'`
     - `'l5_s1'`
   - These levels correspond to the intervertebral spaces between lumbar vertebrae.

2. **Function Definition**:
   - The function `update_row_id` is defined to modify the `row_id` for each row in the DataFrame. It takes two parameters: `row` (the current row of the DataFrame) and `levels` (the list of anatomical levels).
   - Within the function:
     - The anatomical level is determined using `row.name % len(levels)`, which calculates the index based on the current row's position. This ensures a cyclic assignment of levels to the `row_id`.
     - The function returns a new `row_id` formatted as a string that combines the `study_id`, the associated `condition`, and the corresponding anatomical `level`.

3. **Applying the Update**:
   - The `apply()` method is used to update the `row_id` in `test_df` by applying the `update_row_id` function across all rows.
   - The `axis=1` parameter indicates that the function should be applied row-wise.

4. **Viewing the Updated DataFrame**:
   - Finally, the first few rows of the updated DataFrame (`test_df`) are displayed using `test_df.head()`, allowing us to inspect the changes made to the `row_id`.


This section successfully enhances the `row_id` of each entry in the `test_df` DataFrame by incorporating relevant spinal levels. This addition not only improves the identification of conditions but also provides better context for the analysis of imaging data related to lumbar spine conditions.

------------

In [14]:
# Levels for row_id
levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']

# update row_id with levels
def update_row_id(row, levels):
    level = levels[row.name % len(levels)]  
    return f"{row['study_id']}_{row['condition']}_{level}"

# Update row_id in expanded_test_desc to include levels
test_df['row_id'] = test_df.apply(lambda row: update_row_id(row, levels), axis=1)

test_df.head()

,study_id,series_id,series_description,image_path,condition,row_id
0,44036939,2828203845,Sagittal T1,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing_l1_l2
1,44036939,2828203845,Sagittal T1,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing_l2_l3
2,44036939,2828203845,Sagittal T1,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing_l3_l4
3,44036939,2828203845,Sagittal T1,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing_l4_l5
4,44036939,2828203845,Sagittal T1,/kaggle/input/rsna-2024-lumbar-spine-degenerat...,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing_l5_s1


### Loading DICOM Images and Custom Dataset Class

In this section, we define a method for loading DICOM images and create a custom dataset class to facilitate the handling of test data in our machine learning model. This is essential for preparing the images for inference.

#### Loading DICOM Images

1. **Function Definition**:
   - The function `load_dicom` is designed to load an image from a DICOM file given its path.
   - It takes a single parameter, `image_path`, which specifies the location of the DICOM file.

2. **Error Handling**:
   - The function employs a try-except block to manage exceptions that may arise during the loading process.
   - If an error occurs, a `FileNotFoundError` is raised, detailing the nature of the error.

3. **Reading the DICOM File**:
   - The `pydicom.dcmread()` function reads the DICOM file, with the `force=True` argument ensuring that it attempts to read the file even if it does not conform to the standard.

4. **Image Processing**:
   - The pixel data is extracted using `dicom.pixel_array`.
   - If the image data type is not already `uint8`, it is converted to this format to standardize the pixel values.
   - If the image is grayscale (i.e., has two dimensions), it is converted to RGB format by stacking the grayscale channel three times.

5. **Return Statement**:
   - The processed image is returned for further use.

#### Custom Dataset Class

1. **Class Definition**:
   - A custom dataset class named `CustomTestDataset` is defined, inheriting from `torch.utils.data.Dataset`. This allows us to create a dataset compatible with PyTorch data loaders.

2. **Initialization**:
   - The `__init__` method takes two parameters: `dataframe`, which contains the metadata and paths for the images, and an optional `transform` parameter to apply transformations to the images.
   - These parameters are stored as instance variables for use in other methods.

3. **Length Method**:
   - The `__len__` method returns the length of the dataset, which is the number of entries in the DataFrame.

4. **Item Retrieval**:
   - The `__getitem__` method retrieves an image and its associated metadata for a specific index:
     - It uses `iloc` to safely access the image path corresponding to the given index.
     - The `load_dicom` function is called to load the image from the specified path.
     - If a transformation is provided, it is applied to the image.
     - A dictionary called `row_data` is created to hold the metadata associated with the current index (in this case, only `row_id`).

5. **Return Statement**:
   - The method returns a tuple containing the processed image and its associated metadata (`row_data`).


This section provides the functionality to load DICOM images and define a custom dataset class for our test dataset. This setup is crucial for preparing and managing the data needed for model inference, ensuring that the images are correctly loaded and transformed for further analysis.


----------------

In [15]:
import pydicom
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
import torch

def load_dicom(image_path):
    """Load an image from a DICOM file."""
    try:
        dicom = pydicom.dcmread(image_path, force=True)
        image = dicom.pixel_array
        # Convert to uint8 if not already in that format
        if image.dtype != np.uint8:
            image = image.astype(np.uint8)
        # Convert grayscale to RGB if needed
        if len(image.shape) == 2:  # Grayscale image
            image = np.stack([image] * 3, axis=-1)  # Repeat the channel
        return image
    except Exception as e:
        raise FileNotFoundError(f"Error loading DICOM file {image_path}: {str(e)}")

class CustomTestDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        image_path = self.dataframe['image_path'].iloc[index]  # Ensure using iloc for safe access
        image = load_dicom(image_path)
        
        if self.transform:
            image = self.transform(image)
        
         # Create row_data as a dictionary for the current index
        row_data = {
            'row_id': self.dataframe['row_id'].iloc[index],
            # Add other fields as necessary
        }
        return image, row_data

### Image Transformations for Model Input

In this section, we define a series of image transformations to prepare the images for input into our deep learning model. These transformations are essential for ensuring that the images are in the correct format and size, allowing the model to make accurate predictions.

#### Overview of Transformations

1. **Importing the Necessary Library**:
   - The `transforms` module from `torchvision` is imported to leverage various image transformation functions provided by PyTorch.

2. **Transformation Pipeline**:
   - We create a transformation pipeline using `transforms.Compose()`, which allows us to chain multiple transformations together. The transformations applied in the pipeline are as follows:
   
3. **Detailed Explanation of Each Transformation**:
   - **`transforms.ToPILImage()`**:
     - This transformation converts a NumPy array (or a tensor) into a PIL image format. This step is necessary if the input images are in a format that is not directly compatible with the subsequent transformations.
  
   - **`transforms.Resize((224, 224))`**:
     - Resizes the image to dimensions of 224x224 pixels, which is a common input size for many deep learning models, including EfficientNet. This ensures that all images fed into the model have a consistent size, preventing shape mismatches.

   - **`transforms.ToTensor()`**:
     - Converts the PIL image to a PyTorch tensor. This is a crucial step because the model requires input data in tensor format for computation.

   - **`transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])`**:
     - Normalizes the tensor image by adjusting the pixel values to have a mean and standard deviation that are consistent with those of the ImageNet dataset. This normalization helps in improving the model's convergence during training and inference by ensuring that the input data has a standard scale.

4. **Importance of Transformations**:
   - These transformations are essential for preparing the images so that they conform to the expected input specifications of the model. Properly transformed images contribute significantly to the performance of the model by improving its ability to generalize from the training data.



This section effectively outlines the transformations applied to the images before they are fed into the model. By standardizing the input images through resizing, tensor conversion, and normalization, we set the stage for improved model performance and accurate predictions.

------------

In [16]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert NumPy array to PIL image (if necessary)
    transforms.Resize((224, 224)),  # Resize the image to match the model input size
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Apply ImageNet normalization
])


### Creating a Test Dataset and DataLoader

In this section, we create a custom test dataset and a corresponding DataLoader to facilitate the batch processing of test images for inference.

#### Overview

1. **Custom Test Dataset**:
   - We initialize a `CustomTestDataset` object named `test_dataset`. This dataset is responsible for loading the test images and their associated metadata.
   - The parameters passed during initialization include:
     - `test_df`: The DataFrame containing the paths to the images and other metadata.
     - `batch_size`: Set to 8, which specifies the number of samples to be loaded in each batch.
     - `transform`: The previously defined transformation pipeline that processes the images as they are loaded.

### Creating the DataLoader

In this section, we create a `DataLoader` object named `test_loader` to facilitate the iterative loading of test images from the dataset.

#### Purpose of DataLoader

The `DataLoader` is crucial for efficiently handling the data during inference, allowing us to process multiple samples at once and manage batch sizes effectively.

#### Parameters of the DataLoader

1. **`test_dataset`**:
   - This parameter specifies the dataset from which the `DataLoader` will load data. In this case, it is our previously defined `test_dataset`, which contains the test images and their corresponding metadata.

2. **`batch_size`**:
   - The `batch_size` is set to 8, meaning that each iteration of the `DataLoader` will yield 8 images. This allows for efficient processing of the images in manageable groups.

3. **`shuffle`**:
   - The `shuffle` parameter is set to `False`. This means that the data will be loaded in the order it appears in the dataset without any randomization. Maintaining the correct order is important during inference to ensure that predictions can be accurately matched to their respective inputs.



By creating the `test_loader`, we enable the efficient loading of test data, optimizing memory usage and computational efficiency during the inference phase of model evaluation.

-----------

In [17]:
test_dataset = CustomTestDataset(test_df, transform=transform)  

test_loader = DataLoader(test_dataset,batch_size = 8, shuffle=False)

### Loading the Pre-trained Model

In this section, we demonstrate how to load the pre-trained model that has been saved to disk. This process is crucial for performing inference on new data using the model we have trained.

#### Model Loading Process

1. **Importing the Necessary Library**:
   - We import the `torch` library, which provides functionalities for loading and managing models in PyTorch.

2. **Model Path**:
   - We specify the path to the saved model weights. In this case, the model is located at:
     ```python
     model_path = "/kaggle/input/rsna-model/pytorch/default/1/full_model30epochs.pth"
     ```

3. **Loading the Model**:
   - We use `torch.load()` to load the model from the specified path. The `map_location='cpu'` argument ensures that the model is loaded to the CPU, which is particularly useful if the model was trained on a GPU but is being used on a machine without GPU support. 
   - The loaded model is assigned to the variable `unified_model`, which we can use for inference.
  
-------------

### UnifiedEfficientNetV2 Class

In this section, we define the `UnifiedEfficientNetV2` class, which builds upon the EfficientNetV2 architecture. This class is tailored for multi-class classification tasks, utilizing transfer learning while allowing for the customization of the final classification layers.

#### Overview

1. **Class Definition**:
   - The `UnifiedEfficientNetV2` class inherits from `torch.nn.Module` and defines the architecture for our model. 

2. **Constructor (`__init__` Method)**:
   - **Parameters**:
     - `num_classes`: Specifies the number of output classes for classification (default is 3).
     - `pretrained`: A boolean flag indicating whether to load pretrained weights (default is `True`).
     - `weights_path`: A string path to load local weights if available (default is `None`).

   - **Model Initialization**:
     - Loads the EfficientNetV2-S model from the `torchvision` library, optionally using pretrained weights.
     - If a local weights path is provided, it loads those weights into the model.

   - **Classifier Replacement**:
     - Replaces the default classifier of EfficientNetV2 with an identity function. This allows us to define custom classification layers.

   - **Fully Connected Layers**:
     - Defines three fully connected layers (`fc1`, `fc2`, `fc3`) with Batch Normalization and ReLU activation.
     - The output layer (`fc3`) maps to the specified number of classes.

   - **Dropout Layers**:
     - Incorporates dropout layers (`dropout1` and `dropout2`) for regularization to help prevent overfitting during training.

3. **Forward Method**:
   - Defines the forward pass of the network:
     - Extracts embeddings from the EfficientNetV2 model.
     - Passes the embeddings through the fully connected layers and applies dropout as specified.


The `UnifiedEfficientNetV2` class provides a flexible architecture for image classification tasks, leveraging the EfficientNetV2 backbone while allowing customization of the classifier. 

For detailed documentation and explanations of the training process and architecture decisions, please refer to the training notebook, where I have provided a comprehensive overview of each component and its purpose.

-----

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class UnifiedEfficientNetV2(nn.Module):
    def __init__(self, num_classes=3, pretrained=True, weights_path=None):
        super(UnifiedEfficientNetV2, self).__init__()

        # Load the EfficientNetV2 with optional pretrained weights
        self.model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT if pretrained else None)

        # Load local weights if a path is provided
        if weights_path is not None:
            self.model.load_state_dict(torch.load(weights_path))

        # Capture in_features from the last layer of the original classifier
        in_features = self.model.classifier[-1].in_features  # Should be 1280 for EfficientNetV2-S
        
        # Replace the classifier with an identity function (we'll handle classification manually)
        self.model.classifier = nn.Identity()

        # Define fully connected layers with BatchNorm and ReLU activation
        self.fc1 = nn.Sequential(
            nn.Linear(in_features, 256),  # Update input size to 1280
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True)
        )
        self.fc3 = nn.Linear(128, num_classes)

        # Dropout layers for regularization
        self.dropout1 = nn.Dropout(p=0.5)
        self.dropout2 = nn.Dropout(p=0.3)

    def forward(self, x):
        # Get embeddings from EfficientNetV2
        embeddings = self.model(x)  # Should output shape (batch_size, 1280)

        # Fully connected layers with dropout and activations
        x = self.fc1(embeddings)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.fc3(x)

        return x

In [19]:
import torch

# Load the full model

model_path = "/kaggle/input/rsna-model/pytorch/default/1/full_model30epochs.pth" 
unified_model = torch.load(model_path, map_location='cpu')  # Load directly


/tmp/ipykernel_36/2435724777.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unified_model = torch.load(model_path, map_location='cpu')  # Load directly


### Inference on the Test Set

In this section, we perform inference using the pre-trained model on the test dataset. The goal is to predict the probabilities of different conditions for each test image.

#### Setup

1. **Importing Libraries**:
   - We import the necessary libraries, including `torch`, `pandas`, and `tqdm`. The `tqdm` library is used to display a progress bar during batch processing, providing visual feedback on the status of inference.

2. **Setting the Device**:
   - We determine whether to run the model on a GPU or CPU. If a CUDA-capable GPU is available, we use it; otherwise, we fall back to the CPU. This is done using:
     ```python
     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
     ```

3. **Model Preparation**:
   - We ensure the model is set to evaluation mode by calling `unified_model.eval()`. This is crucial as it changes the behavior of certain layers (like dropout and batch normalization) to behave appropriately during inference.

#### Inference Process

1. **Initializing Results Dictionary**:
   - We create a dictionary named `results` to store the output probabilities for each condition corresponding to each image. This dictionary includes:
     - `row_id`: Unique identifier for each test image.
     - `normal_mild`: Probability of the condition being normal or mild.
     - `moderate`: Probability of the condition being moderate.
     - `severe`: Probability of the condition being severe.

2. **Inference Loop**:
   - Using a `with torch.no_grad()` context manager, we disable gradient calculation to reduce memory usage and improve performance during inference.
   - We loop through the test DataLoader (`test_loader`), processing images in batches. The `tqdm` function provides a progress bar to indicate the batch processing status.

3. **Processing Batches**:
   - For each batch of images:
     - We transfer the images to the specified device (CPU or GPU) using `images.to(device)`.
     - The model outputs predictions for the images using `unified_model(images)`.
     - We apply the softmax function to the output logits to convert them into probabilities for each class, storing the results in `probs`.

4. **Storing Results**:
   - For each image in the batch, we extract the corresponding `row_id` and the predicted probabilities for each condition (normal/mild, moderate, severe). These values are appended to the results dictionary.

#### Normalization and Validation

1. **Creating a DataFrame**:
   - After processing all batches, we convert the results dictionary into a pandas DataFrame named `results_df`.

2. **Normalizing Probabilities**:
   - To ensure that the predicted probabilities for each condition sum to 1, we normalize the probabilities by dividing each probability by the sum of probabilities for each image.

3. **Validation**:
   - We check for any invalid (negative) probabilities in the DataFrame. If any negative probabilities are found, a `ValueError` is raised, indicating a potential issue in the inference process.


This section demonstrates how to perform inference on a test dataset using the pre-trained model. By processing images in batches, we efficiently compute the probabilities for different conditions while ensuring the results are valid and properly normalized for further analysis or submission.

----

In [20]:
import torch
import pandas as pd
from tqdm import tqdm
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming model is already loaded and set to eval mode
unified_model.eval()

# Initialize a dictionary to store the results
results = {
    'row_id': [],
    'normal_mild': [],
    'moderate': [],
    'severe': []
}

print("Starting inference on the test set...")

with torch.no_grad():
    for batch_idx, (images, row_data) in enumerate(tqdm(test_loader, desc="Processing batches")):
        images = images.to(device)

        outputs = unified_model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()

        for i in range(len(probs)):
            row_id = row_data['row_id'][i]
            results['row_id'].append(row_id)
            results['normal_mild'].append(probs[i][0])  # Probability for 'Normal/Mild'
            results['moderate'].append(probs[i][1])     # Probability for 'Moderate'
            results['severe'].append(probs[i][2])       # Probability for 'Severe'

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Normalize probabilities to ensure they sum to 1
results_df[['normal_mild', 'moderate', 'severe']] = results_df[['normal_mild', 'moderate', 'severe']].div(
    results_df[['normal_mild', 'moderate', 'severe']].sum(axis=1), axis=0
)

# Check for any invalid values
if (results_df[['normal_mild', 'moderate', 'severe']] < 0).any().any():
    raise ValueError("Found negative probabilities in submission.")

results_df = pd.DataFrame(results)

Starting inference on the test set...


Processing batches: 100%|██████████| 25/25 [00:29<00:00,  1.17s/it]


### Overview of the Results DataFrame

After generating the predictions, we utilize the `info()` method on the `results_df` DataFrame to obtain a summary of its structure and contents. This step is crucial for understanding the data we have produced during the inference process.

#### Key Aspects of `results_df.info()`:

1. **DataFrame Summary:**
   - The `info()` method provides a concise summary of the DataFrame, including:
     - The number of entries (rows) and columns.
     - The index range.
     - The data types of each column.
     - The count of non-null entries in each column.

2. **Column Breakdown:**
   - The DataFrame consists of the following columns:
     - `row_id`: Unique identifier for each image, linking it to its respective condition and study.
     - `normal_mild`: Probability score for the image being classified as normal or mild.
     - `moderate`: Probability score indicating a moderate classification.
     - `severe`: Probability score suggesting a severe classification.

3. **Data Types:**
   - The data types of the columns will typically include:
     - `row_id`: Object (string type).
     - `normal_mild`, `moderate`, `severe`: Float (probability values).

4. **Non-null Counts:**
   - The count of non-null entries helps verify that there are no missing values in the predictions, ensuring the integrity of the results.

5. **Importance of the Summary:**
   - By examining this information, we can confirm that the predictions have been successfully generated and structured correctly. It also helps identify any issues, such as missing values or incorrect data types, before proceeding with further analysis or visualizations.

Overall, running `results_df.info()` is an important step to validate the output of our model and prepare for subsequent tasks like analysis, visualization, or reporting of the results.


In [21]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   row_id       194 non-null    object 
 1   normal_mild  194 non-null    float32
 2   moderate     194 non-null    float32
 3   severe       194 non-null    float32
dtypes: float32(3), object(1)
memory usage: 3.9+ KB


### Averaging Results per `row_id`

In this section, we aggregate the prediction results for each unique `row_id` by calculating the average probabilities across the corresponding images. This process is essential for consolidating the predictions to reflect the overall assessment for each condition associated with a specific study.

#### Key Steps:

1. **Grouping by `row_id`:**
   - We use the `groupby()` method on the `results_df` DataFrame to group the data based on the `row_id`. This ensures that we consolidate predictions for each unique image identifier.

   - The `as_index=False` parameter is set to maintain `row_id` as a regular column in the resulting DataFrame instead of using it as an index.

2. **Calculating Mean Probabilities:**
   - For each `row_id`, we compute the mean of the probability scores for the classes: `normal_mild`, `moderate`, and `severe`. This results in a new DataFrame called `averaged_results_df`, which contains the average probability scores for each `row_id`.

3. **Normalizing Probabilities:**
   - To ensure that the probabilities across the three classes sum to 1 for each `row_id`, we perform normalization:
     - We calculate the sum of the probabilities for each `row_id` using `sum(axis=1)`.
     - Each probability column (`normal_mild`, `moderate`, `severe`) is divided by the corresponding sum of probabilities. This step ensures that the predicted probabilities are properly scaled.

4. **Checking for Invalid Values:**
   - We perform a validation check to ensure that there are no negative probability values in the normalized probabilities. This is critical, as probabilities must always be non-negative and fall within the range [0, 1].
   - If any negative probabilities are found, a `ValueError` is raised, indicating that the submission contains invalid values.

5. **Importance of Averaging and Normalization:**
   - Averaging results across multiple predictions for the same `row_id` provides a more robust estimate of the true condition.
   - Normalizing the probabilities is essential for interpretation, as it allows for direct comparisons among the different conditions.

This step is crucial for preparing the final prediction results, ensuring they are valid and ready for analysis, visualization, or submission.


In [22]:
# Average results per row_id
averaged_results_df = results_df.groupby('row_id', as_index=False).mean()

# Normalize probabilities to ensure they sum to 1
sum_probs = averaged_results_df[['normal_mild', 'moderate', 'severe']].sum(axis=1)
averaged_results_df['normal_mild'] = averaged_results_df['normal_mild'] / sum_probs
averaged_results_df['moderate'] = averaged_results_df['moderate'] / sum_probs
averaged_results_df['severe'] = averaged_results_df['severe'] / sum_probs

# Check for any invalid values
if (averaged_results_df[['normal_mild', 'moderate', 'severe']] < 0).any().any():
    raise ValueError("Found negative probabilities in submission.")

### Verification of Probability Normalization

In this step, we verify that the normalized probabilities for each condition sum to 1 for every `row_id` in the `averaged_results_df` DataFrame. This check ensures that the normalization process was successful and that the probability values are valid.

#### Key Steps:

1. **Calculating the Sum of Probabilities:**
   - We create a new column called `sum_check` in the `averaged_results_df` DataFrame. This column will hold the sum of the probabilities for the three conditions: `normal_mild`, `moderate`, and `severe`.
   - The sum is calculated using the `sum(axis=1)` method, which sums the values across the specified columns for each row.

2. **Rounding the Sum Values:**
   - To enhance readability and ensure consistency, we apply the `round(x, 2)` function to round the sum to two decimal places. This rounding helps in clearly observing the results without excessive precision that may not be meaningful in the context of probabilities.

3. **Displaying the Normalization Check:**
   - We print a confirmation message "Normalization Check:" to indicate that we are about to display the results of the verification process.
   - The `head()` method is used to show the first few entries of the `row_id` along with their corresponding `sum_check` values. This display allows us to quickly assess whether the normalization was successful across several samples.

4. **Importance of the Normalization Check:**
   - Ensuring that the sum of probabilities equals 1 is crucial for validating the model's outputs. Probabilities must adhere to the properties of a probability distribution, where the sum of all possible outcomes should equal 1.
   - This check serves as a final validation step before proceeding to utilize or submit the results, providing confidence in the integrity of the data.

By performing this verification, we confirm that our predictions are reliable and ready for further analysis or reporting.


In [23]:
# Verify that the sum of probabilities is 1 for each row
averaged_results_df['sum_check'] = averaged_results_df[['normal_mild', 'moderate', 'severe']].sum(axis=1).apply(lambda x: round(x, 2))
print("Normalization Check:")
print(averaged_results_df[['row_id', 'sum_check']])

Normalization Check:
                                             row_id  sum_check
0    44036939_left_neural_foraminal_narrowing_l1_l2        1.0
1    44036939_left_neural_foraminal_narrowing_l2_l3        1.0
2    44036939_left_neural_foraminal_narrowing_l3_l4        1.0
3    44036939_left_neural_foraminal_narrowing_l4_l5        1.0
4    44036939_left_neural_foraminal_narrowing_l5_s1        1.0
5         44036939_left_subarticular_stenosis_l1_l2        1.0
6         44036939_left_subarticular_stenosis_l2_l3        1.0
7         44036939_left_subarticular_stenosis_l3_l4        1.0
8         44036939_left_subarticular_stenosis_l4_l5        1.0
9         44036939_left_subarticular_stenosis_l5_s1        1.0
10  44036939_right_neural_foraminal_narrowing_l1_l2        1.0
11  44036939_right_neural_foraminal_narrowing_l2_l3        1.0
12  44036939_right_neural_foraminal_narrowing_l3_l4        1.0
13  44036939_right_neural_foraminal_narrowing_l4_l5        1.0
14  44036939_right_neural_foramina

#### Interpretation of Results

- **Consistent Results**: Each `row_id` shows a `sum_check` value of **1.0**. This indicates that the probabilities for `normal/mild`, `moderate`, and `severe` conditions have been successfully normalized.
  
- **Validity of Probabilities**: The normalization ensures that the output probabilities are valid and reflect the model's confidence in its predictions.


The normalization check confirms that the predictions for each `row_id` are valid, reinforcing the reliability of the model's output. This step is critical before proceeding to the submission or further analysis of the results

-----

### Preparing the Submission DataFrame

In this section, we create a new DataFrame specifically designed for submission purposes. This DataFrame will contain the final average probability results for each unique `row_id`, allowing for straightforward analysis or submission.

#### Key Steps:

1. **Creating the Submission DataFrame:**
   - We define `submission_df` by selecting specific columns from the `averaged_results_df`. The columns included are:
     - `row_id`: The unique identifier for each image, linking it to its respective condition and study.
     - `normal_mild`: The averaged probability score indicating the likelihood that the condition is classified as normal or mild.
     - `moderate`: The averaged probability score representing a moderate classification.
     - `severe`: The averaged probability score suggesting a severe classification.

   This selection ensures that only the relevant data for submission is retained in the new DataFrame.

2. **Structure of the Submission DataFrame:**
   - The resulting `submission_df` will have the following columns:
     - **row_id**: A string that uniquely identifies each row.
     - **normal_mild**: A float representing the predicted probability of the normal/mild condition.
     - **moderate**: A float representing the predicted probability of the moderate condition.
     - **severe**: A float representing the predicted probability of the severe condition.

3. **Viewing the Submission DataFrame:**
   - By calling `submission_df`, we can preview the structure and contents of the DataFrame. This step is essential for verifying that the data has been correctly organized and is ready for further processing, such as exporting to a CSV file for submission or reporting.

4. **Importance of the Submission DataFrame:**
   - The `submission_df` serves as the final output of our inference process. It consolidates all the predictions into a format that can be easily interpreted, shared, or submitted to relevant stakeholders or platforms.
   - Ensuring that the data is structured correctly is crucial for effective communication of the model’s findings and performance on the test dataset.

This preparation step is vital for the completion of the project, marking the transition from model inference to result dissemination.


In [24]:
submission_df = averaged_results_df[['row_id', 'normal_mild', 'moderate', 'severe']]
submission_df

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.341481,0.369015,0.289504
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.361994,0.371449,0.266557
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.357654,0.379326,0.263020
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.392068,0.372919,0.235013
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.377461,0.363348,0.259191
5,44036939_left_subarticular_stenosis_l1_l2,0.403106,0.356870,0.240024
6,44036939_left_subarticular_stenosis_l2_l3,0.398327,0.358709,0.242964
7,44036939_left_subarticular_stenosis_l3_l4,0.431757,0.327438,0.240805
8,44036939_left_subarticular_stenosis_l4_l5,0.458173,0.308042,0.233786
9,44036939_left_subarticular_stenosis_l5_s1,0.327154,0.358067,0.314780


### Saving the Submission File

In this final step, we save the prepared submission DataFrame (`submission_df`) to a CSV file. This file can be used for further analysis, sharing, or submission to relevant platforms.

#### Key Steps:

1. **Exporting the DataFrame to CSV:**
   - We utilize the `to_csv()` method from the pandas library to export `submission_df` to a CSV file.
   - The argument `index=False` is specified to prevent pandas from writing row indices to the CSV file. This is important because we want to keep the file clean and ensure it only contains the relevant data columns.

2. **Naming the Submission File:**
   - The submission file is named `submission.csv`, and this name is indicated in the print statement for clarity. The CSV file will contain the `row_id` and the corresponding averaged probability scores for the conditions: normal/mild, moderate, and severe.

3. **Outputting a Confirmation Message:**
   - A confirmation message is printed to the console to inform the user that the submission file has been successfully saved. This provides assurance that the data has been correctly exported.

4. **Saving to Kaggle Working Directory:**
   - The command `submission_df.to_csv('/kaggle/working/submission.csv', index=False)` ensures that the file is saved to the Kaggle working directory, making it accessible for download or further use within the Kaggle environment.

5. **Importance of Saving the Submission File:**
   - Saving the predictions in a structured format (CSV) is essential for effective communication of results.
   - The submission file can be submitted to competitions or assessments where model performance needs to be evaluated.
   - This step marks the conclusion of the data processing and inference pipeline, transitioning from analysis to actionable outcomes.

By completing this step, we ensure that our results are preserved and ready for future reference or evaluation.

------------

In [25]:
submission_df.to_csv('submission.csv', index=False)
# Save the submission file
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
print("Submission file saved as 'submission.csv'.")

Submission file saved as 'submission.csv'.


### Conclusion

In this notebook, we successfully implemented the inference process for the RSNA Lumbar Spine Degenerative Classification task using a pre-trained Convolutional Neural Network (CNN). The key steps undertaken include:

1. **Data Preparation:** 
   - We loaded and processed the test dataset, ensuring that all images were correctly accessed and organized.

2. **Model Loading and Predictions:** 
   - We utilized a pre-trained CNN model to make predictions on the test dataset. The model output was carefully handled to ensure accurate results.

3. **Results Aggregation and Normalization:** 
   - We computed the average probabilities for each condition across multiple images corresponding to each `row_id`, ensuring that the probabilities were normalized to sum to 1.

4. **Verification:** 
   - A final check confirmed that the normalized probabilities adhered to the fundamental properties of probability distributions.

5. **Submission Preparation:**
   - The processed results were saved in a structured CSV format, ready for submission or further analysis.

Through these steps, we have built a robust inference pipeline that can be adapted for similar classification tasks in medical imaging. The results generated can be valuable for clinical evaluations and decision-making regarding lumbar spine conditions. Future work may focus on improving model accuracy through fine-tuning and exploring additional data augmentation techniques.

## Thank you for reviewing this notebook, and I look forward to any questions or feedback!

-----------